In [1]:
import numpy as np
import tensorflow as tf


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:

# get data from train_1.csv
tmp = np.genfromtxt("train_1.csv", dtype=float, delimiter=",")

tem = tmp[1:,2:]
ave = np.mean(tem,axis = 0)*np.ones((891,6))
ptp = np.ptp(tem,axis = 0)*np.ones((891,6))
tem = np.subtract(tem,ave)
tem = np.true_divide(tem,ptp)
tmp[1:,2:] = tem

train_data = tmp[1:601,2:]
validation_data0 = tmp[601:,2:]

train_lable = tmp[1:601,1]
validation_lable0 = tmp[601:,1]
train_lable = train_lable.reshape(600,1)
validation_lable0 = validation_lable0.reshape(291,1)

def add_layer(input,in_size,out_size):
    theta = tf.Variable(tf.random_normal([in_size,out_size]))
    if out_size==1:
        z = tf.matmul(input,theta)
        a = tf.sigmoid(z)
    else:
        z = tf.matmul(input,theta)
        a = tf.sigmoid(z)
        a_0 = tf.ones([600,1])
        a = tf.concat([a_0,a],1)
    return theta,a,z


# define placeholder for inputs to network
xs = tf.placeholder(tf.float32,[None,6])
ys = tf.placeholder(tf.float32,[None,1])

# add hidden layer
theta_1,l1,z_1 = add_layer(xs,6,12)
# add output layer
theta_2,output,z_2 = add_layer(l1,13,1)

#regularzation
lamda = 0.05
regularzation = (tf.reduce_sum(tf.square(theta_1))+tf.reduce_sum(tf.square(theta_2)))*lamda/600

# the error between prediction and real data
J_theta = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = ys,logits = z_2)) + regularzation

a = 3 # learning rate
train_step = tf.train.GradientDescentOptimizer(a).minimize(J_theta)

#prediction
validation_data = tf.constant(validation_data0,tf.float32)
validation_lable = tf.constant(validation_lable0,tf.float32)

a_1 = tf.sigmoid(tf.matmul(validation_data,theta_1))
a_0 = tf.ones([291,1])
a_20 = tf.concat([a_0,a_1],1)
a_2 = tf.sigmoid(tf.matmul(a_20,theta_2))
prediction = tf.round(a_2)

validation = tf.subtract(prediction,validation_lable)
validation = tf.abs(validation)
accuracy =tf.reduce_sum(validation) 
accuracy = (291-accuracy)/291

#initialize
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(1000000):
    #training
    sess.run(train_step,feed_dict={xs:train_data,ys:train_lable})
    
    if i%1000 == 0:
        loss = sess.run(J_theta,feed_dict={xs:train_data,ys:train_lable})
        print("J_theta:",loss)
        acc = sess.run(accuracy)
        print(acc)
        




J_theta: 0.9678353
0.63573885
J_theta: 0.46860513
0.7869416
J_theta: 0.46634302
0.7938144
J_theta: 0.465491
0.7938144
J_theta: 0.46495357
0.7938144
J_theta: 0.46458304
0.7938144
J_theta: 0.464357
0.7938144
J_theta: 0.46420988
0.7938144
J_theta: 0.46410766
0.7938144
J_theta: 0.4640241
0.7938144
J_theta: 0.46384016
0.7938144
J_theta: 0.46276242
0.7938144
J_theta: 0.46130323
0.8006873
J_theta: 0.46070805
0.8006873
J_theta: 0.46047014
0.8041237
J_theta: 0.4603592
0.8041237
J_theta: 0.46029034
0.8041237
J_theta: 0.46023235
0.8041237
J_theta: 0.46017194
0.8041237
J_theta: 0.46010447
0.8041237
J_theta: 0.4600339
0.8041237
J_theta: 0.45997155
0.8041237
J_theta: 0.4599264
0.8041237
J_theta: 0.4598986
0.8041237
J_theta: 0.45988283
0.8041237
J_theta: 0.459874
0.8041237
J_theta: 0.4598687
0.8041237
J_theta: 0.45986527
0.8041237
J_theta: 0.45986283
0.8041237
J_theta: 0.45986086
0.8041237


KeyboardInterrupt: 